In [ ]:
from flask import Flask, jsonify
import pandas as pd
import yfinance as yf
from datetime import datetime
import requests
from sec_edgar_downloader import Downloader

app = Flask(__name__)
CACHE = {}
CACHE_TTL = 15 * 60
SEC_DOWNLOADER = Downloader("./sec_filings")


def get_sp500_tickers():
    table = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
    return table[0]["Symbol"].tolist()


def fetch_from_sec(ticker):
    try:
        headers = {"User-Agent": "stock-screener educational app"}

        cik_lookup = requests.get(
            "https://www.sec.gov/files/company_tickers.json",
            headers=headers
        ).json()

        cik = None
        for v in cik_lookup.values():
            if v["ticker"] == ticker:
                cik = str(v["cik_str"]).zfill(10)
                break
        if not cik:
            return None

        facts = requests.get(
            f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json",
            headers=headers
        ).json()

        def get_latest(tag, namespace="us-gaap"):
            items = facts.get("facts", {}).get(namespace, {}).get(tag, {}).get("units", {})
            for unit in items.values():
                return unit[-1].get("val")
            return None

        net_income = get_latest("NetIncomeLoss")
        total_assets = get_latest("Assets")
        equity = get_latest("StockholdersEquity")
        interest_expense = get_latest("InterestExpense")
        ebit = get_latest("OperatingIncomeLoss")
        total_liabilities = get_latest("Liabilities")

        roa = net_income / total_assets if net_income and total_assets else None
        roe = net_income / equity if net_income and equity else None
        ebit_interest = ebit / abs(interest_expense) if ebit and interest_expense else None
        de_ratio = total_liabilities / equity if total_liabilities and equity else None

        return {
            "roa": roa,
            "roe": roe,
            "ebit_interest": ebit_interest,
            "de_ratio": de_ratio
        }

    except Exception:
        return None

        net_income = extract_number('NetIncomeLoss')
        total_assets = extract_number('Assets')
        equity = extract_number('StockholdersEquity')
        ebit = extract_number('EBIT')
        interest_expense = extract_number('InterestExpense')
        total_debt = extract_number('Liabilities')
        revenue = extract_number('Revenue')

        roa = net_income / total_assets if net_income and total_assets else None
        roe = net_income / equity if net_income and equity else None
        ebit_interest = ebit / abs(interest_expense) if ebit and interest_expense else None
        de_ratio = total_debt / equity if total_debt and equity else None
        current_ratio = None  # Could parse current assets / current liabilities if available
        revenue_growth = None  # Could compare with prior year filing

        return {
            "roa": roa,
            "roe": roe,
            "ebit_interest": ebit_interest,
            "de_ratio": de_ratio,
            "current_ratio": current_ratio,
            "revenue_growth": revenue_growth
        }

    except Exception:
        return None


def fetch_fundamentals(ticker):
    current_year = datetime.now().year
    sec_data = fetch_from_sec(ticker)

    try:
        stock = yf.Ticker(ticker)
        info = stock.info or {}
    except Exception:
        info = {}

    ipo_date = info.get("ipoDate")
    ipo_year = int(ipo_date[:4]) if ipo_date else current_year - 7

    price = info.get("currentPrice")
    volume = info.get("volume")
    one_year_return = info.get("52WeekChange")

    if sec_data:
        roa = sec_data.get("roa")
        roe = sec_data.get("roe")
        ebit_interest = sec_data.get("ebit_interest")
        de_ratio = sec_data.get("de_ratio")
        current_ratio = sec_data.get("current_ratio")
        revenue_growth = sec_data.get("revenue_growth")
    else:
        roa = info.get("returnOnAssets")
        roe = info.get("returnOnEquity")
        ebit_interest = info.get("ebitda") / info.get("interestExpense") if info.get("interestExpense") else None
        de_ratio = info.get("debtToEquity")
        current_ratio = None
        revenue_growth = None

    return {
        "symbol": ticker,
        "price": price,
        "volume": volume,
        "one_year_return": one_year_return,
        "roa": roa,
        "roe": roe,
        "ebit_interest": ebit_interest,
        "de_ratio": de_ratio,
        "current_ratio": current_ratio,
        "revenue_growth": revenue_growth,
        "age": current_year - ipo_year,
    }


def run_screening_pipeline():
    now = datetime.now().timestamp()

    cached = CACHE.get("top10")
    if cached and now - cached["time"] < CACHE_TTL:
        return cached["data"]

    tickers = get_sp500_tickers()
    data = []

    for t in tickers:
        f = fetch_fundamentals(t)
        if not f or f["age"] < 7:
            continue
        data.append(f)

    df = pd.DataFrame(data)

    df["roa_rank"] = df["roa"].rank(pct=True)
    df["roe_rank"] = df["roe"].rank(pct=True)
    df["ebit_interest_rank"] = df["ebit_interest"].rank(pct=True)
    df["de_rank"] = (1 / df["de_ratio"]).rank(pct=True)
    df["current_ratio_rank"] = df["current_ratio"].rank(pct=True)
    df["revenue_growth_rank"] = df["revenue_growth"].rank(pct=True)
    df["return_rank"] = df["one_year_return"].rank(pct=True)

    weights = {
        "roa_rank": 0.20,
        "roe_rank": 0.20,
        "ebit_interest_rank": 0.20,
        "de_rank": 0.15,
        "current_ratio_rank": 0.10,
        "revenue_growth_rank": 0.15
    }

    weighted_sum = sum(weights[m] * df[m] for m in weights)
    df["quality_score"] = weighted_sum / sum(weights.values())

    top10 = df.sort_values("quality_score", ascending=False).head(10)

    result = top10[["symbol", "price", "one_year_return", "volume", "quality_score"]].to_dict(orient="records")
    CACHE["top10"] = {"time": now, "data": result}

    return result


@app.route("/top10")
def top10():
    return jsonify(run_screening_pipeline())
